### Final Results With Comparison to Different Model Performance

In [2]:
!pip install xgboost


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
print(pd.__version__)
import numpy as np
import json
import logging
import xgboost as xgb
import ast
import itertools
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2
import os
import sys

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

2.1.3


In [4]:
sys.path.append('/root/Unified_LN_V2/')
from Utilities import *

In [5]:
def air_calc_new(pred,prefix):
    air = {}
    target_control_dict = [
            {'age_cuts': {
                'target': ['Age 62 and over'],
                'control': ['Age below 62']
            }},
            {'Race': {
                'target': ['black', 'hispanic'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['black'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['hispanic'],
                'control': ['white']
            }},
            {'Gender': {
                'target': ['F'],
                'control': ['M']
            }},
            # 'mmct_tag': {
            #     'target': ['mmct'],
            #     'control': ['non-mmct']
            # }
    ]

    for var in target_control_dict:
        for i in var:
            target_list = var[i]['target']
            control_list = var[i]['control']
            p_target_list= ','.join(target_list) if (
                (len(target_list)<= 1) and (('black' in target_list) or ('hispanic' in target_list))
            ) else ''
            
            ttd_target= pred[pred[i].isin(target_list)]['con_wt'].sum()
            approved_target= pred[pred[i].isin(target_list)]['approved'].multiply(pred[pred[i].isin(target_list)]['con_wt']).sum()
            ttd_control= pred[pred[i].isin(control_list)]['con_wt'].sum()
            approved_control= pred[pred[i].isin(control_list)]['approved'].multiply(pred[pred[i].isin(control_list)]['con_wt']).sum()
            app_pct_target= round(approved_target/ttd_target, 4)
            app_pct_control= round(approved_control/ttd_control, 4)
            air[prefix+i+p_target_list]= [round(app_pct_target/app_pct_control, 4)]
        
            # print({'variable': i, 'target_list': p_target_list, 'ttd_target': ttd_target, 'ttd_control': ttd_control, 
                   # 'approved_target': approved_target, 'approved_control': approved_control})
            
    # print(air)
        
    return air

In [6]:
col_tuple_pass= ["unique_id", 'client', "wt2", 'fico_pb', 'auto_score', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts']

In [7]:
def get_risk_rank(data, target, cols_tuple, score_col= 'p1', weight= 'RECWGT', asc= False):
    dd = data.loc[:, cols_tuple + [score_col]]
    # dd = pd.DataFrame(dd)
    # dd = dd.reset_index()
    dd['RECWGT'] = dd["wt2"]
    dd['conwt'] = 1
    dd[target] = dd[target] 
    # pred = pd.concat([dd.loc[:, cols_tuple + ['RECWGT', 'conwt']],p],axis=1)
    pred= dd.copy()
    # display(pred.head())
    print(pred[weight].sum(),((pred[weight]*pred[target]).sum()),pred[weight].max())

    data_ks_cap5= rnkOrderingN(pred, target, weight, score_col, 5, asc)
    # display(data_ks_cap5[0])
    # print(data_ks_cap5[1])
    # data_ks_cap10= rnkOrderingN(pred, target, weight, score_col, 10, asc)
    # display(data_ks_cap10[0])
    # print(data_ks_cap10[1])
    return data_ks_cap5

In [8]:
def get_segment(d):
    if (d['derogcount']> 0):
        return 'Derog'
    elif ((d['derogcount']== 0) and (d['assetprop']> 0)):
        return 'Property Owner'
    elif ((d['derogcount']== 0) and (d['assetprop']== 0)):
        return 'Non Property Owner'
    else:
        return 'Missing'

In [9]:
def Equal_Approval_Cutoff_Barsath(pred, target, score_col, asc= True):
    pred= pred.copy()
    actual_approved_count= pred[target].sum()
    pred= pred.sort_values(by= [score_col], ascending= asc)
    pred['con_wt']= 1
    pred['cumsum']= pred['con_wt'].cumsum()
    pred['approved']= np.where(pred['cumsum']<= actual_approved_count, 1, 0)
    return pred #cutoff, original_dataset

## Reading Scored Data - Overall LDA

In [10]:
path_overall= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/With_LDA/Iteration_3/' #/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/With_LDA/
os.listdir(path_overall)

['With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 '050424_LN_Auto_KGB_LDA_No_Segment_Hopt_4.csv',
 'With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'xgbm_ln_auto_v2_hpt_best_with_lda_3.zip',
 'With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv']

In [11]:
# train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-03-29.csv")
# ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-03-29.csv")

# train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-02.csv")
# ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-02.csv")

train_scored_all= pd.read_csv(f"{path_overall}With_LDA_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv")
test_scored_all= pd.read_csv(f"{path_overall}With_LDA_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv")
oot_scored_all= pd.read_csv(f"{path_overall}With_LDA_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv")
ttd_scored_all= pd.read_csv(f"{path_overall}With_LDA_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv")

train_scored_all.shape, test_scored_all.shape, oot_scored_all.shape, ttd_scored_all.shape

((146855, 12), (62938, 12), (37121, 12), (685215, 12))

In [12]:
funded_scored_all= pd.concat([train_scored_all, test_scored_all, oot_scored_all]).reset_index(drop= True)
funded_scored_all.shape, ttd_scored_all.shape, ttd_scored_all['funded_flag'].sum()

((246914, 12), (685215, 12), 200148.0)

In [13]:
funded_scored_all['unique_id']= funded_scored_all['unique_id'].astype(str)
ttd_scored_all['unique_id']= ttd_scored_all['unique_id'].astype(str)

In [14]:
funded_scored_all.rename(columns= {'p1': 'p1_overall_lda'}, inplace= True)
ttd_scored_all.rename(columns= {'p1': 'p1_overall_lda'}, inplace= True)

### Reading V 1.5 Scored Data

In [15]:
v15= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Data/040424_Auto_LN_V1_5_Scored_Data.csv')
v15.head()

,unique_id,client,p1_v1_5,fico_pb,dataset
0,453208,SECU,0.053346,624.0,fico_lt_660
1,453394,SECU,0.064319,0.0,fico_lt_660
2,453515,SECU,0.086988,614.0,fico_lt_660
3,453555,SECU,0.035332,637.0,fico_lt_660
4,453594,SECU,0.094630,587.0,fico_lt_660


In [16]:
v15.shape, ttd_scored_all.shape, funded_scored_all.shape

((824212, 5), (685215, 12), (246914, 12))

In [17]:
v15['unique_id']= v15['unique_id'].astype(str)

In [18]:
print(ttd_scored_all.shape)
ttd_scored_all= ttd_scored_all.merge(v15[['unique_id', 'client', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
ttd_scored_all.shape

(685215, 12)


(685215, 13)

In [19]:
ttd_scored_all['p1_v1_5'].isnull().sum()

0

In [20]:
ttd_scored_all[['p1_v1_5', 'p1_overall_lda']].describe()

,p1_v1_5,p1_overall_lda
count,685215.000000,685215.000000
mean,0.038407,0.021863
std,0.032355,0.022076
min,0.007456,0.005289
25%,0.014268,0.007273
50%,0.025966,0.015229
75%,0.052640,0.024805
max,0.190158,0.114281


In [21]:
print(funded_scored_all.shape)
funded_scored_all= funded_scored_all.merge(v15[['unique_id', 'client', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
funded_scored_all.shape, funded_scored_all['p1_v1_5'].isnull().sum()

(246914, 12)


((246914, 13), 0)

In [22]:
# rnkOrderingN(funded_scored_all[~(funded_scored_all['fico_pb']>= 740)], 'bad_flag', None, 'p1_overall_lda', 5)

In [23]:
# rnkOrderingN(funded_scored_all[~(funded_scored_all['fico_pb']>= 740)], 'bad_flag', None, 'p1_v1_5', 5)

#### Reading FICO Segment Data

In [24]:
path_owner= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Auto_FICO_lt_660_Model/'
os.listdir(path_owner)

['FICO_lt_660_050424_LN_Auto_KGB_LDA_Hopt_2.csv',
 'Mono_Data',
 'Data',
 '.ipynb_checkpoints',
 'FICO_lt_660_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'xgbm_ln_auto_v2_hpt_best_FICO_lt_660.zip',
 'FICO_lt_660_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'FICO_lt_660_050424_LN_Auto_KGB_LDA_Hopt_3.csv',
 'FICO_lt_660_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'FICO_lt_660_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'FICO_lt_660_030424_LN_Auto_KGB_LDA_Hopt_1.csv']

In [25]:
train_scored_owner= pd.read_csv(f'{path_owner}FICO_lt_660_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv')
test_scored_owner= pd.read_csv(f'{path_owner}FICO_lt_660_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv')
oot_scored_owner= pd.read_csv(f'{path_owner}FICO_lt_660_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv')
ttd_scored_owner= pd.read_csv(f'{path_owner}FICO_lt_660_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv')
train_scored_owner.shape, test_scored_owner.shape, oot_scored_owner.shape, ttd_scored_owner.shape

((26240, 12), (11247, 12), (6668, 12), (259792, 12))

In [26]:
train_scored_owner['unique_id']= train_scored_owner['unique_id'].astype(str)
test_scored_owner['unique_id']= test_scored_owner['unique_id'].astype(str)
oot_scored_owner['unique_id']= oot_scored_owner['unique_id'].astype(str)
ttd_scored_owner['unique_id']= ttd_scored_owner['unique_id'].astype(str)

In [27]:
train_scored_owner['dataset']= 'FICO_LT_660'
test_scored_owner['dataset']= 'FICO_LT_660'
oot_scored_owner['dataset']= 'FICO_LT_660'
ttd_scored_owner['dataset']= 'FICO_LT_660'

#### Reading 660-740

In [28]:
path_non_owner= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Auto_FICO_660_740_Model/'
os.listdir(path_non_owner)

['FICO_660_740_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'xgbm_ln_auto_v2_hpt_best_FICO_660_740.zip',
 'FICO_660_740_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'Mono_Data',
 'Data',
 'FICO_660_740_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'FICO_660_740_030424_LN_Auto_KGB_LDA_Hopt_1.csv',
 '.ipynb_checkpoints',
 'FICO_660_740_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv',
 'FICO_660_740_050424_LN_Auto_KGB_LDA_Hopt_2.csv']

In [29]:
train_scored_nonowner= pd.read_csv(f'{path_non_owner}FICO_660_740_Train_Auto_LN_V2_Scored_Data_2024-04-08.csv')
test_scored_nonowner= pd.read_csv(f'{path_non_owner}FICO_660_740_Test_Auto_LN_V2_Scored_Data_2024-04-08.csv')
oot_scored_nonowner= pd.read_csv(f'{path_non_owner}FICO_660_740_Oot_Auto_LN_V2_Scored_Data_2024-04-08.csv')
ttd_scored_nonowner= pd.read_csv(f'{path_non_owner}FICO_660_740_TTD_Auto_LN_V2_Scored_Data_2024-04-08.csv')
train_scored_nonowner.shape, test_scored_nonowner.shape, oot_scored_nonowner.shape, ttd_scored_nonowner.shape

((53526, 12), (22940, 12), (13665, 12), (228747, 12))

In [30]:
train_scored_nonowner['unique_id']= train_scored_nonowner['unique_id'].astype(str)
test_scored_nonowner['unique_id']= test_scored_nonowner['unique_id'].astype(str)
oot_scored_nonowner['unique_id']= oot_scored_nonowner['unique_id'].astype(str)
ttd_scored_nonowner['unique_id']= ttd_scored_nonowner['unique_id'].astype(str)

In [31]:
train_scored_nonowner['dataset']= 'FICO_660_740'
test_scored_nonowner['dataset']= 'FICO_660_740'
oot_scored_nonowner['dataset']= 'FICO_660_740'
ttd_scored_nonowner['dataset']= 'FICO_660_740'

In [32]:
train_ono= pd.concat([train_scored_owner, train_scored_nonowner]).reset_index(drop= True)
test_ono= pd.concat([test_scored_owner, test_scored_nonowner]).reset_index(drop= True)
oot_ono= pd.concat([oot_scored_owner, oot_scored_nonowner]).reset_index(drop= True)
ttd_ono= pd.concat([ttd_scored_owner, ttd_scored_nonowner]).reset_index(drop= True)
train_ono.shape, test_ono.shape, oot_ono.shape, ttd_ono.shape

((79766, 13), (34187, 13), (20333, 13), (488539, 13))

In [33]:
print(train_ono['dataset'].value_counts(dropna= False), train_scored_owner.shape, train_scored_nonowner.shape)
print(test_ono['dataset'].value_counts(dropna= False), test_scored_owner.shape, test_scored_nonowner.shape)
print(oot_ono['dataset'].value_counts(dropna= False), oot_scored_owner.shape, oot_scored_nonowner.shape)
print(ttd_ono['dataset'].value_counts(dropna= False), ttd_scored_owner.shape, ttd_scored_nonowner.shape)

dataset
FICO_660_740    53526
FICO_LT_660     26240
Name: count, dtype: int64 (26240, 13) (53526, 13)
dataset
FICO_660_740    22940
FICO_LT_660     11247
Name: count, dtype: int64 (11247, 13) (22940, 13)
dataset
FICO_660_740    13665
FICO_LT_660      6668
Name: count, dtype: int64 (6668, 13) (13665, 13)
dataset
FICO_LT_660     259792
FICO_660_740    228747
Name: count, dtype: int64 (259792, 13) (228747, 13)


In [34]:
del train_scored_all, train_scored_nonowner, train_scored_owner
del test_scored_all, test_scored_nonowner, test_scored_owner
del oot_scored_all, oot_scored_nonowner, oot_scored_owner
del ttd_scored_nonowner, ttd_scored_owner

In [35]:
import gc
gc.collect()

0

### Merging p1-overall

In [36]:
print(train_ono.shape)
train_ono= train_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(train_ono.shape)
print(train_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(test_ono.shape)
test_ono= test_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(test_ono.shape)
print(test_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(oot_ono.shape)
oot_ono= oot_ono.merge(funded_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(oot_ono.shape)
print(oot_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

print(ttd_ono.shape)
ttd_ono= ttd_ono.merge(ttd_scored_all[['unique_id', 'client', 'p1_overall_lda', 'p1_v1_5']], on= ['unique_id', 'client'], how= 'left')
print(ttd_ono.shape)
print(ttd_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].isnull().sum())

(79766, 13)
(79766, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(34187, 13)
(34187, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(20333, 13)
(20333, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64
(488539, 13)
(488539, 15)
p1                0
p1_overall_lda    0
p1_v1_5           0
dtype: int64


In [37]:
ttd_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), train_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), test_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe(), oot_ono[['p1', 'p1_overall_lda', 'p1_v1_5']].describe()

(                  p1  p1_overall_lda        p1_v1_5
 count  488539.000000   488539.000000  488539.000000
 mean        0.036216        0.025858       0.047088
 std         0.029825        0.024149       0.034185
 min         0.003063        0.005289       0.007456
 25%         0.012328        0.008925       0.020267
 50%         0.027645        0.018214       0.037432
 75%         0.051672        0.028622       0.064244
 max         0.232617        0.114281       0.190158,
                  p1  p1_overall_lda       p1_v1_5
 count  79766.000000    79766.000000  79766.000000
 mean       0.026323        0.021353      0.036112
 std        0.024877        0.020708      0.029133
 min        0.003063        0.005289      0.007456
 25%        0.008373        0.007453      0.015088
 50%        0.017102        0.016376      0.025590
 75%        0.036325        0.024709      0.048128
 max        0.214624        0.114281      0.188945,
                  p1  p1_overall_lda       p1_v1_5
 count  341

In [38]:
train_ono.groupby(['dataset'])[['p1', 'p1_overall_lda', 'p1_v1_5']].apply(lambda x: x.describe()).T

dataset        FICO_660_740                                                    \
                      count      mean       std       min       25%       50%   
p1                  53526.0  0.013589  0.009805  0.003063  0.006863  0.010642   
p1_overall_lda      53526.0  0.017774  0.017027  0.005289  0.007080  0.012634   
p1_v1_5             53526.0  0.022418  0.014222  0.007456  0.012809  0.018149   

dataset                            FICO_LT_660                                \
                     75%       max       count      mean       std       min   
p1              0.017309  0.102884     26240.0  0.052299  0.026071  0.019454   
p1_overall_lda  0.021837  0.114281     26240.0  0.028653  0.025155  0.005289   
p1_v1_5         0.027091  0.160300     26240.0  0.064044  0.031699  0.020207   

dataset                                                 
                     25%       50%       75%       max  
p1              0.032982  0.046852  0.064449  0.214624  
p1_overall_lda  0.012432  0.020237  0.030570  0.114281  
p1_v1_5         0.040836  0.055724  0.081057  0.188945

In [39]:
train_ono[train_ono['dataset'].isin(['FICO_LT_660'])].shape

(26240, 15)

In [40]:
train_ono['dataset'].value_counts(dropna= False)

dataset
FICO_660_740    53526
FICO_LT_660     26240
Name: count, dtype: int64

In [41]:
def get_model_comparison_segment(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a

In [42]:
def get_model_comparison_overall(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_overall_lda', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1_overall_lda'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a
    

In [43]:
def get_model_comparison_v15(train_d, test_d, oot_d, ttd_d, segment, pref, d):
    train_d= train_d.copy()
    test_d= test_d.copy()
    oot_d= oot_d.copy()
    ttd_d= ttd_d.copy()
    print({'segment': segment})#, 'overall shape': data.shape, 'segmented shape': data[data['dataset'].isin(segment)].shape})
    
    a= {'Model': pref, 'Dataset': d}
    
    a[f'KS Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', 'wt2', 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Train Model']= rnkOrderingN(train_d[train_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Test Model']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    a[f'KS Non Weighted Oot Model']= rnkOrderingN(oot_d[oot_d['dataset'].isin(segment)], 'bad_flag', None, 'p1_v1_5', n_bkts= 5, asc= False)[1]['KS']
    
    a[f'KS Non Weighted Oot FICO']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'fico_pb', n_bkts= 5, asc= True)[1]['KS']
    a[f'KS Non Weighted Oot Auto Score']= rnkOrderingN(test_d[test_d['dataset'].isin(segment)], 'bad_flag', None, 'auto_score', n_bkts= 5, asc= True)[1]['KS']
    
    a['Bad Rate Train']= train_d[train_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Test']= test_d[test_d['dataset'].isin(segment)]['bad_flag'].mean()
    a['Bad Rate Oot']= oot_d[oot_d['dataset'].isin(segment)]['bad_flag'].mean()
    
    air_sgement_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'p1_v1_5'), 'AIR TTD')
    a[f'AIR Age Model']= air_sgement_model['AIR TTDage_cuts']
    a[f'AIR Race Model']= air_sgement_model['AIR TTDRace']
    a[f'AIR Race Black Model']= air_sgement_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic Model']= air_sgement_model['AIR TTDRacehispanic']
    a[f'AIR Gender Model']= air_sgement_model['AIR TTDGender']
    
    air_fico_model= air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_d[ttd_d['dataset'].isin(segment)], 'approve_flag', 'fico_pb', asc= False), 'AIR TTD')
    a[f'AIR Age FICO']= air_fico_model['AIR TTDage_cuts']
    a[f'AIR Race FICO']= air_fico_model['AIR TTDRace']
    a[f'AIR Race Black FICO']= air_fico_model['AIR TTDRaceblack']
    a[f'AIR Race Hispanic FICO']= air_fico_model['AIR TTDRacehispanic']
    a[f'AIR Gender FICO']= air_fico_model['AIR TTDGender']
    
    return a

In [44]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_LT_660'], 'FICO_LT_660', 'FICO_LT_660')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_LT_660'], 'Overall', 'FICO_LT_660')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_LT_660'], 'V1.5', 'FICO_LT_660'))]
)

{'segment': ['FICO_LT_660']}
{'segment': ['FICO_LT_660']}
{'segment': ['FICO_LT_660']}


,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,FICO_LT_660,FICO_LT_660,24.36,23.14,24.69,23.73,24.98,15.14,23.37,0.050762,0.050325,0.047241,1.8436,0.7523,0.7805,0.7073,0.9526,1.1556,0.8307,0.7166,1.0111,0.976
0,Overall,FICO_LT_660,20.60,23.10,21.53,21.83,23.64,15.14,23.37,0.050762,0.050325,0.047241,1.7243,0.7914,0.7855,0.8010,0.9329,1.1556,0.8307,0.7166,1.0111,0.976
0,V1.5,FICO_LT_660,22.12,21.07,20.97,23.36,23.98,15.14,23.37,0.050762,0.050325,0.047241,1.7936,0.8010,0.8197,0.7716,0.9740,1.1556,0.8307,0.7166,1.0111,0.976


In [45]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_660_740'], 'FICO_660_740', 'FICO_660_740')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_660_740'], 'Overall', 'FICO_660_740')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_660_740'], 'V1.5', 'FICO_660_740'))]
)

{'segment': ['FICO_660_740']}
{'segment': ['FICO_660_740']}
{'segment': ['FICO_660_740']}


,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,FICO_660_740,FICO_660_740,25.76,25.68,25.65,25.64,24.31,18.21,17.89,0.013676,0.013688,0.013611,1.2570,0.8285,0.9041,0.7679,0.9719,1.0684,0.935,0.9128,0.9527,0.9741
0,Overall,FICO_660_740,24.32,23.19,23.71,23.38,25.40,18.21,17.89,0.013676,0.013688,0.013611,1.3118,0.8691,0.9386,0.8135,0.9568,1.0684,0.935,0.9128,0.9527,0.9741
0,V1.5,FICO_660_740,25.07,21.97,26.48,23.70,24.86,18.21,17.89,0.013676,0.013688,0.013611,1.2711,0.8175,0.9432,0.7167,0.9796,1.0684,0.935,0.9128,0.9527,0.9741


In [46]:
pd.concat(
    [pd.DataFrame(get_model_comparison_segment(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_LT_660', 'FICO_660_740'], 'Both', 'Both')),
     pd.DataFrame(get_model_comparison_overall(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_LT_660', 'FICO_660_740'], 'Overall', 'Both')),
     pd.DataFrame(get_model_comparison_v15(train_ono, test_ono, oot_ono, ttd_ono, ['FICO_LT_660', 'FICO_660_740'], 'V1.5', 'Both'))]
)

{'segment': ['FICO_LT_660', 'FICO_660_740']}
{'segment': ['FICO_LT_660', 'FICO_660_740']}
{'segment': ['FICO_LT_660', 'FICO_660_740']}


,Model,Dataset,KS Weighted Train Model,KS Weighted Test Model,KS Non Weighted Train Model,KS Non Weighted Test Model,KS Non Weighted Oot Model,KS Non Weighted Oot FICO,KS Non Weighted Oot Auto Score,Bad Rate Train,Bad Rate Test,Bad Rate Oot,AIR Age Model,AIR Race Model,AIR Race Black Model,AIR Race Hispanic Model,AIR Gender Model,AIR Age FICO,AIR Race FICO,AIR Race Black FICO,AIR Race Hispanic FICO,AIR Gender FICO
0,Both,Both,37.36,34.55,36.18,34.53,32.87,32.31,27.99,0.025876,0.025741,0.02464,1.3774,0.7607,0.6680,0.8716,0.9481,1.1889,0.7914,0.6473,0.9639,0.9455
0,Overall,Both,28.32,28.32,27.68,29.86,28.97,32.31,27.99,0.025876,0.025741,0.02464,1.5280,0.7946,0.7838,0.8078,0.9321,1.1889,0.7914,0.6473,0.9639,0.9455
0,V1.5,Both,36.28,35.44,35.68,34.88,32.25,32.31,27.99,0.025876,0.025741,0.02464,1.4358,0.7528,0.6850,0.8343,0.9547,1.1889,0.7914,0.6473,0.9639,0.9455


In [47]:
rnkOrderingN(train_ono, 'bad_flag', None, 'p1', 5)

(   min_score  max_score  mean_score  total    Goods    Bads  Bad_Rate  \
 0   0.042880   0.214624    0.067578  15954  14863.0  1091.0      6.84   
 1   0.022532   0.042877    0.031333  15953  15491.0   462.0      2.90   
 2   0.012751   0.022532    0.017377  15953  15691.0   262.0      1.64   
 3   0.007256   0.012751    0.009761  15953  15793.0   160.0      1.00   
 4   0.003063   0.007256    0.005566  15953  15864.0    89.0      0.56   
 
    Cum_Total  Cum_Good  Cum_Bad     KS  
 0       20.0     19.13    52.86  33.73  
 1       40.0     39.06    75.24  36.18  
 2       60.0     59.26    87.94  28.68  
 3       80.0     79.58    95.69  16.11  
 4      100.0    100.00   100.00   0.00  ,
 {'Gini': 45.879999999999995, 'KS': 36.18})

In [55]:
ttd_ono['dataset'].value_counts(dropna= False)

dataset
FICO_LT_660     259792
FICO_660_740    228747
Name: count, dtype: int64

In [59]:
ttd_ono.groupby('dataset').apply(lambda x: air_calc_new(Equal_Approval_Cutoff_Barsath(x, 'approve_flag', 'p1'), 'AIR TTD'))

dataset
FICO_660_740     {'AIR TTDage_cuts': [1.257], 'AIR TTDRace': [0.8285], 'AIR TTDRaceblack': [0.9041], 'AIR TTDRacehispanic': [0.7679], 'AIR TTDGender': [0.9719]}
FICO_LT_660     {'AIR TTDage_cuts': [1.8436], 'AIR TTDRace': [0.7523], 'AIR TTDRaceblack': [0.7805], 'AIR TTDRacehispanic': [0.7073], 'AIR TTDGender': [0.9526]}
dtype: object

In [52]:
air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono, 'approve_flag', 'p1_v1_5'), 'AIR TTD')

{'AIR TTDage_cuts': [1.4358],
 'AIR TTDRace': [0.7528],
 'AIR TTDRaceblack': [0.685],
 'AIR TTDRacehispanic': [0.8343],
 'AIR TTDGender': [0.9547]}

In [53]:
air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono, 'approve_flag', 'p1'), 'AIR TTD')

{'AIR TTDage_cuts': [1.3774],
 'AIR TTDRace': [0.7607],
 'AIR TTDRaceblack': [0.668],
 'AIR TTDRacehispanic': [0.8716],
 'AIR TTDGender': [0.9481]}

In [54]:
air_calc_new(Equal_Approval_Cutoff_Barsath(ttd_ono, 'approve_flag', 'p1_overall_lda'), 'AIR TTD')

{'AIR TTDage_cuts': [1.528],
 'AIR TTDRace': [0.7946],
 'AIR TTDRaceblack': [0.7838],
 'AIR TTDRacehispanic': [0.8078],
 'AIR TTDGender': [0.9321]}

### END

In [51]:
train_ono.groupby('dataset')['fico_pb'].agg(['min', 'max'])

,min,max
dataset,,
FICO_660_740,660.0,740.0
FICO_LT_660,0.0,659.0


In [60]:
def air_calc_new_2(pred,prefix):
    air = {}
    target_control_dict = [
            {'age_cuts': {
                'target': ['Age 62 and over'],
                'control': ['Age below 62']
            }},
            {'Race': {
                'target': ['black', 'hispanic'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['black'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['hispanic'],
                'control': ['white']
            }},
            {'Gender': {
                'target': ['F'],
                'control': ['M']
            }},
            # 'mmct_tag': {
            #     'target': ['mmct'],
            #     'control': ['non-mmct']
            # }
    ]

    for var in target_control_dict:
        for i in var:
            target_list = var[i]['target']
            control_list = var[i]['control']
            p_target_list= ','.join(target_list) if (
                (len(target_list)<= 1) and (('black' in target_list) or ('hispanic' in target_list))
            ) else ''
            
            ttd_target= pred[pred[i].isin(target_list)]['con_wt'].sum()
            approved_target= pred[pred[i].isin(target_list)]['approved'].multiply(pred[pred[i].isin(target_list)]['con_wt']).sum()
            ttd_control= pred[pred[i].isin(control_list)]['con_wt'].sum()
            approved_control= pred[pred[i].isin(control_list)]['approved'].multiply(pred[pred[i].isin(control_list)]['con_wt']).sum()
            app_pct_target= round(approved_target/ttd_target, 4)
            app_pct_control= round(approved_control/ttd_control, 4)
            air[prefix+i+p_target_list]= [round(app_pct_target/app_pct_control, 4)]
        
            print({'variable': i, 'target_list': p_target_list, 'ttd_target': ttd_target, 'ttd_control': ttd_control, 
                   'approved_target': approved_target, 'approved_control': approved_control})
            
    print(air)
        
    return air

In [61]:
air_calc_new_2(Equal_Approval_Cutoff_Barsath(ttd_ono, 'approve_flag', 'p1'), 'AIR TTD')

{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 40529, 'ttd_control': 445884, 'approved_target': 26255, 'approved_control': 209679}
{'variable': 'Race', 'target_list': '', 'ttd_target': 95125, 'ttd_control': 259333, 'approved_target': 38550, 'approved_control': 138164}
{'variable': 'Race', 'target_list': 'black', 'ttd_target': 51856, 'ttd_control': 259333, 'approved_target': 18457, 'approved_control': 138164}
{'variable': 'Race', 'target_list': 'hispanic', 'ttd_target': 43269, 'ttd_control': 259333, 'approved_target': 20093, 'approved_control': 138164}
{'variable': 'Gender', 'target_list': '', 'ttd_target': 191167, 'ttd_control': 257696, 'approved_target': 90833, 'approved_control': 129119}
{'AIR TTDage_cuts': [1.3774], 'AIR TTDRace': [0.7607], 'AIR TTDRaceblack': [0.668], 'AIR TTDRacehispanic': [0.8716], 'AIR TTDGender': [0.9481]}


{'AIR TTDage_cuts': [1.3774],
 'AIR TTDRace': [0.7607],
 'AIR TTDRaceblack': [0.668],
 'AIR TTDRacehispanic': [0.8716],
 'AIR TTDGender': [0.9481]}

In [62]:
ttd_ono.groupby('dataset').apply(lambda x: air_calc_new_2(Equal_Approval_Cutoff_Barsath(x, 'approve_flag', 'p1'), 'AIR TTD'))

{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 22375, 'ttd_control': 206372, 'approved_target': 17763, 'approved_control': 130338}
{'variable': 'Race', 'target_list': '', 'ttd_target': 38155, 'ttd_control': 131888, 'approved_target': 21699, 'approved_control': 90529}
{'variable': 'Race', 'target_list': 'black', 'ttd_target': 16984, 'ttd_control': 131888, 'approved_target': 10540, 'approved_control': 90529}
{'variable': 'Race', 'target_list': 'hispanic', 'ttd_target': 21171, 'ttd_control': 131888, 'approved_target': 11159, 'approved_control': 90529}
{'variable': 'Gender', 'target_list': '', 'ttd_target': 87379, 'ttd_control': 124765, 'approved_target': 56228, 'approved_control': 82610}
{'AIR TTDage_cuts': [1.257], 'AIR TTDRace': [0.8285], 'AIR TTDRaceblack': [0.9041], 'AIR TTDRacehispanic': [0.7679], 'AIR TTDGender': [0.9719]}
{'variable': 'age_cuts', 'target_list': '', 'ttd_target': 18154, 'ttd_control': 239512, 'approved_target': 10723, 'approved_control': 76741}
{'variable

dataset
FICO_660_740     {'AIR TTDage_cuts': [1.257], 'AIR TTDRace': [0.8285], 'AIR TTDRaceblack': [0.9041], 'AIR TTDRacehispanic': [0.7679], 'AIR TTDGender': [0.9719]}
FICO_LT_660     {'AIR TTDage_cuts': [1.8436], 'AIR TTDRace': [0.7523], 'AIR TTDRaceblack': [0.7805], 'AIR TTDRacehispanic': [0.7073], 'AIR TTDGender': [0.9526]}
dtype: object

### Misc

In [3]:
pwd

'/root/Unified_LN_V2/LN_Auto_V2/Code'

In [9]:
ls '/usr/share/'

ImageMagick-6/    dict/                  icons/             perl/
X11/              djvu/                  info/              perl5/
adduser/          doc/                   kerberos-configs/  pixmaps/
applications/     doc-base/              keyrings/          pkgconfig/
apport/           dpkg/                  libc-bin/          polkit-1/
base-files/       emacs/                 libthai/           poppler/
base-passwd/      fontconfig/            lintian/           python3/
bash-completion/  fonts/                 locale/            readline/
binfmts/          fonts-droid-fallback/  man/               sensible-utils/
bug/              gcc/                   menu/              tabset/
ca-certificates/  gdb/                   mercurial/         terminfo/
color/            ghostscript/           metainfo/          ucf/
common-licenses/  git-core/              misc/              xml/
dbus-1/           gitweb/                netpbm/            zoneinfo/
debconf/          glib-2.0/        